In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, trim, regexp_replace, lower, concat, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import pandas as pd

import os
from pathlib import Path

project_dir = Path(os.getcwd()).parent
data_dir = project_dir / 'data'
raw_data_dir = data_dir / 'raw'
cleaned_data_dir = data_dir / 'cleaned'

## Data Wrangling

In [2]:
print("File Name", '\t\t\t\t', 'Size in GB')
for file in raw_data_dir.glob('*.json'):
    print(file.name, ' \t', round(os.path.getsize(file) / 1e9, 2), 'GB')

File Name 				 Size in GB
yelp_academic_dataset_business.json  	 0.12 GB
yelp_academic_dataset_checkin.json  	 0.29 GB
yelp_academic_dataset_review.json  	 5.34 GB
yelp_academic_dataset_tip.json  	 0.18 GB
yelp_academic_dataset_user.json  	 3.36 GB


In [3]:
# Create a SparkSession object with the desired configuration options
spark = SparkSession.builder \
    .appName("YelpDataEda") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.executor.cores", "2") \
    .config("spark.driver.maxResultSize", "1g") \
    .getOrCreate()

### Wrangling Businesses Data (yelp_academic_dataset_business.json)

In [4]:
business_schema = StructType([
    StructField("business_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("address", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True),
    StructField("postal_code", StringType(), True),
    StructField("latitude", StringType(), True),
    StructField("longitude", StringType(), True),
    StructField("stars", StringType(), True),
    StructField("review_count", IntegerType(), True),
    StructField("is_open", IntegerType(), True),
    StructField("categories", StringType(), True),
    StructField("attributes", StructType([
        StructField("Alcohol", StringType(), True),
        StructField("Ambience", StructType([
            StructField("casual", StringType(), True),
            StructField("classy", StringType(), True),
            StructField("divey", StringType(), True),
            StructField("hipster", StringType(), True),
            StructField("intimate", StringType(), True),
            StructField("romantic", StringType(), True),
            StructField("touristy", StringType(), True),
            StructField("trendy", StringType(), True),
            StructField("upscale", StringType(), True)
        ]), True),
        StructField("BikeParking", StringType(), True),
        StructField("BusinessAcceptsCreditCards", StringType(), True),
        StructField("BusinessParking", StructType([
            StructField("garage", StringType(), True),
            StructField("lot", StringType(), True),
            StructField("street", StringType(), True),
            StructField("valet", StringType(), True)
        ]), True),
        StructField("GoodForKids", StringType(), True),
        StructField("HasTV", StringType(), True),
        StructField("NoiseLevel", StringType(), True),
        StructField("OutdoorSeating", StringType(), True),
        StructField("RestaurantsAttire", StringType(), True),
        StructField("RestaurantsDelivery", StringType(), True),
        StructField("RestaurantsGoodForGroups", StringType(), True),
        StructField("RestaurantsPriceRange2", StringType(), True),
        StructField("RestaurantsReservations", StringType(), True),
        StructField("RestaurantsTakeOut", StringType(), True),
        StructField("WiFi", StringType(), True)
    ]), True),
    StructField("hours", StructType([
        StructField("Monday", StringType(), True),
        StructField("Tuesday", StringType(), True),
        StructField("Wednesday", StringType(), True),
        StructField("Thursday", StringType(), True),
        StructField("Friday", StringType(), True),
        StructField("Saturday", StringType(), True),
        StructField("Sunday", StringType(), True)
    ]), True)
])

In [5]:
business_df = spark.read.json(str(raw_data_dir / 'yelp_academic_dataset_business.json'), schema=business_schema)

In [6]:
# Sample of the business_df
# business_df = business_df.sample(False, 0.01)

In [7]:
# Extract attributes
business_df = business_df.withColumn("attr_alcohol", business_df.attributes.Alcohol) \
       .withColumn("attr_bike_parking", business_df.attributes.BikeParking) \
       .withColumn("attr_business_accepts_credit_cards", business_df.attributes.BusinessAcceptsCreditCards) \
       .withColumn("attr_good_for_kids", business_df.attributes.GoodforKids) \
       .withColumn("attr_has_tv", business_df.attributes.HasTV) \
       .withColumn("attr_noise_level", business_df.attributes.NoiseLevel) \
       .withColumn("attr_outdoor_seating", business_df.attributes.OutdoorSeating) \
       .withColumn("attr_restaurants_attire", business_df.attributes.RestaurantsAttire) \
       .withColumn("attr_restaurants_delivery", business_df.attributes.RestaurantsDelivery) \
       .withColumn("attr_restaurants_good_for_groups", business_df.attributes.RestaurantsGoodforGroups) \
       .withColumn("attr_restaurants_price_range2", business_df.attributes.RestaurantsPriceRange2) \
       .withColumn("attr_restaurants_reservations", business_df.attributes.RestaurantsReservations) \
       .withColumn("attr_restaurants_takeout", business_df.attributes.RestaurantsTakeOut) \
       .withColumn("attr_wifi", business_df.attributes.WiFi) \
       .withColumn("attr_ambience_casual", business_df.attributes.Ambience.casual) \
       .withColumn("attr_ambience_classy", business_df.attributes.Ambience.classy) \
       .withColumn("attr_ambience_divey", business_df.attributes.Ambience.divey) \
       .withColumn("attr_ambience_hipster", business_df.attributes.Ambience.hipster) \
       .withColumn("attr_ambience_intimate", business_df.attributes.Ambience.intimate) \
       .withColumn("attr_ambience_romantic", business_df.attributes.Ambience.romantic) \
       .withColumn("attr_ambience_touristy", business_df.attributes.Ambience.touristy) \
       .withColumn("attr_ambience_trendy", business_df.attributes.Ambience.trendy) \
       .withColumn("attr_ambience_upscale", business_df.attributes.Ambience.upscale) \
       .withColumn("attr_business_parking_garage", business_df.attributes.BusinessParking.garage) \
       .withColumn("attr_business_parking_lot", business_df.attributes.BusinessParking.lot) \
       .withColumn("attr_business_parking_street", business_df.attributes.BusinessParking.street) \
       .withColumn("attr_business_parking_valet", business_df.attributes.BusinessParking.valet)

# Extract hours
business_df = business_df.withColumn("hours_monday", business_df.hours.Monday) \
       .withColumn("hours_tuesday", business_df.hours.Tuesday) \
       .withColumn("hours_wednesday", business_df.hours.Wednesday) \
       .withColumn("hours_thursday", business_df.hours.Thursday) \
       .withColumn("hours_friday", business_df.hours.Friday) \
       .withColumn("hours_saturday", business_df.hours.Saturday) \
       .withColumn("hours_sunday", business_df.hours.Sunday)

# Drop original nested columns
business_df = business_df.drop("attributes", "hours")

In [8]:
# Long-form to Wide-form on categories column
business_df.select("categories").show(5, False) # Current categories column

+----------------------------------------------------------------------------------------------------------+
|categories                                                                                                |
+----------------------------------------------------------------------------------------------------------+
|Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists|
|Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services                            |
|Department Stores, Shopping, Fashion, Home & Garden, Electronics, Furniture Stores                        |
|Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries                                                     |
|Brewpubs, Breweries, Food                                                                                 |
+----------------------------------------------------------------------------------------------------------+
only showing top 5 

In [9]:
business_df = business_df.withColumn("category", split("categories", ", "))

# Explode categories column
exploded_categories = business_df.select("business_id", "category", "categories").withColumn("category", explode("category"))
exploded_categories = exploded_categories.withColumn("category", regexp_replace("category", "&", " "))
exploded_categories = exploded_categories.withColumn("category", regexp_replace("category", " +", " "))
exploded_categories = exploded_categories.withColumn("category", regexp_replace("category", " ", "_"))
exploded_categories = exploded_categories.withColumn("category", regexp_replace("category", "-", "_"))
exploded_categories = exploded_categories.withColumn("category", regexp_replace("category", "_+", "_"))
exploded_categories = exploded_categories.withColumn("category", regexp_replace("category", "'", ""))
exploded_categories = exploded_categories.withColumn("category", regexp_replace("category", "&", "and"))
exploded_categories = exploded_categories.withColumn("category", regexp_replace("category", "/", "_or_"))
exploded_categories = exploded_categories.withColumn("category", regexp_replace("category", "\(", ""))
exploded_categories = exploded_categories.withColumn("category", regexp_replace("category", "\)", ""))
exploded_categories = exploded_categories.withColumn("category", trim("category"))
exploded_categories = exploded_categories.withColumn("category", lower("category"))
exploded_categories = exploded_categories.withColumn("category", concat(lit("cat_"), exploded_categories.category))

In [10]:
exploded_categories.select("business_id", "category", "categories").show(10, False) # Exploded categories column

+----------------------+--------------------------------+----------------------------------------------------------------------------------------------------------+
|business_id           |category                        |categories                                                                                                |
+----------------------+--------------------------------+----------------------------------------------------------------------------------------------------------+
|Pns2l4eNsfO8kk83dixA6A|cat_doctors                     |Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists|
|Pns2l4eNsfO8kk83dixA6A|cat_traditional_chinese_medicine|Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists|
|Pns2l4eNsfO8kk83dixA6A|cat_naturopathic_or_holistic    |Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists|
|Pns2l4eNs

In [11]:
# Pivot the resulting rows into columns
pivoted_df = exploded_categories.groupBy("business_id").pivot("category").count().na.fill(0) \

# Join the pivoted DataFrame back to the original DataFrame
joined_df = business_df.join(pivoted_df, "business_id", "left") \
    .drop("categories", "category")

In [12]:
business_pdf = joined_df.toPandas()

# Checks to see if rows count match
(business_df.count(), business_pdf.shape[0])

(150346, 150346)

In [13]:
display(business_pdf.head())
business_pdf.to_csv(str(cleaned_data_dir / 'business.csv'), index=False)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,cat_wine_tasting_room,cat_wine_tours,cat_wineries,cat_womens_clothing,cat_workers_compensation_law,cat_wraps,cat_yelp_events,cat_yoga,cat_ziplining,cat_zoos
0,---kPU91CF4Lq2-WlRu9Lw,Frankie's Raw Bar,4903 State Rd 54,New Port Richey,FL,34652,28.2172884,-82.7333444,4.5,24,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,--LC8cIrALInl2vyo701tg,Studio G Salon,6537 Gunn Hwy,Tampa,FL,33625,28.0658857,-82.5593005,5.0,8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,--gJkxbsiSIwsQKbiwm_Ng,Salon Lofts - West Kennedy,4545 W Kennedy Blvd,Tampa,FL,33609,27.9451223,-82.5210814,5.0,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,--hF_3v1JmU9nlu4zfXJ8Q,Green District Salads - Monument Circle,28 Monument Cir,Indianapolis,IN,46204,39.7678876,-86.1583509,4.5,15,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,--qLiYw2ErSmvVwumb2kdw,C.C.'s Kitchen,517 Station Ave,Haddon Heights,NJ,08035,39.8829629,-75.0580346,5.0,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Free up memory
del business_pdf
del business_df
del joined_df
del exploded_categories
del pivoted_df

### Wrangling Review Data (yelp_academic_dataset_review.json)

In [15]:
# Read the review data from JSON file chunk by chunk
review_df = pd.read_json(str(raw_data_dir / 'yelp_academic_dataset_review.json'), lines=True, chunksize=1_000_000)

In [16]:
# Save the review data to CSV file chunk by chunk
for i, chunk in enumerate(review_df):
    print(f'Processing chunk {i+1}')

    if i == 0:
        chunk.to_csv(str(cleaned_data_dir / 'review.csv'), mode='w', header=True, index=False)
    else:
        chunk.to_csv(str(cleaned_data_dir / 'review.csv'), mode='a', header=False, index=False)

Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 6
Processing chunk 7


In [17]:
del review_df
del chunk

### Wrangling User Data (yelp_academic_dataset_user.json)

In [18]:
# Read the user data from JSON file chunk by chunk
user_df = pd.read_json(str(raw_data_dir / 'yelp_academic_dataset_user.json'), lines=True, chunksize=1_000_000)

In [19]:
# Save the user data to CSV file chunk by chunk
for i, chunk in enumerate(user_df):
    print(f'Processing chunk {i+1}')

    if i == 0:
        chunk.to_csv(str(cleaned_data_dir / 'user.csv'), mode='w', header=True, index=False)
    else:
        chunk.to_csv(str(cleaned_data_dir / 'user.csv'), mode='a', header=False, index=False)

Processing chunk 1
Processing chunk 2


In [20]:
del user_df
del chunk